Scrappping pour mettre à jour les Peak ccu / Peak_players

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

def get_peak_players_from_steamcharts(appid):
    """Retourne le 24h Peak pour un jeu Steam via SteamCharts."""
    
    url = f"https://steamcharts.com/app/{appid}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"❌ Page non trouvée pour AppID {appid}")
            return None
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Chercher la ligne "Last 30 Days"
        rows = soup.find_all("tr")
        
        for row in rows:
            # Vérifier si c'est la ligne "Last 30 Days"
            month_cell = row.find("td", class_="month-cell")
            if month_cell and "Last 30 Days" in month_cell.text:
                # Extraire la valeur du Peak (dernière colonne <td class="right num italic">)
                peak_cell = row.find("td", class_="right num italic")
                if peak_cell:
                    peak_value = int(peak_cell.text.replace(",", "").strip())
                    print(f"✔ AppID {appid} → Peak Players (Last 30 Days): {peak_value}")
                    return peak_value
        
        print(f"⚠ Aucun peak détecté pour AppID {appid}")
        return None
    
    except Exception as e:
        print(f"🔥 Erreur pour AppID {appid}: {e}")
        return None

# Test
if __name__ == "__main__":
    # 
    peak = get_peak_players_from_steamcharts(2246340)
    print(f"\nRésultat final: {peak}")

In [ ]:
# Important du dataset jeux_confirmes
jeux_confirmes = pd.read_csv(r"data/nettoyes/jeux_confirmes.csv")
# Affichage du dataset jeux_confirmes
jeux_confirmes.head()

In [ ]:
import pandas as pd
import time

# Chemin du fichier
output_path = r"C:\Users\bongu\Documents\GAMEDATA360\data\nettoyes\jeux_confirmes_peak_complet.csv"

# 1. Chargement
try:
    df_work = pd.read_csv(output_path)
    print("🔄 Chargement du fichier existant...")
except FileNotFoundError:
    df_work = pd.read_csv(r"C:\Users\bongu\Documents\GAMEDATA360\data\nettoyes\jeux_confirmes.csv")
    print("🆕 Chargement du fichier original...")

# On s'assure que la colonne existe
if 'Peak CCU' not in df_work.columns:
    df_work['Peak CCU'] = None

total_jeux = len(df_work)
print(f"🚀 Démarrage de la mise à jour FORCÉE pour {total_jeux} jeux.")

# 2. Boucle sur TOUS les jeux
for i, (index, row) in enumerate(df_work.iterrows(), 1):
    
    appid = row['AppID']
    
    # --- SCRAPING SYSTÉMATIQUE (Pas de condition if) ---
    peak_players = get_peak_players_from_steamcharts(appid)
    
    # Mise à jour directe
    df_work.at[index, 'Peak CCU'] = peak_players
    
    # Affichage de la progression
    print(f"[{i} / {total_jeux}] 🔄 Mis à jour AppID {appid} -> Peak: {peak_players}")
    
    time.sleep(1) # Pause indispensable
    
    # 3. Sauvegarde intermédiaire (tous les 10 jeux)
    if i % 10 == 0:
        df_work.to_csv(output_path, index=False)
        print(f"💾 Sauvegarde intermédiaire...")

# 4. Sauvegarde Finale
df_work.to_csv(output_path, index=False)
print("✅ Terminé ! Tous les jeux ont été mis à jour.")

In [ ]:
# # Boucle pour scraper les id dans le dataframe jeux_confirmes la colonne 'AppID'
# for index, row in jeux_confirmes.iterrows():
#     appid = row['AppID']
#     peak_players = get_peak_players_from_steamcharts(appid)
#     jeux_confirmes.at[index, 'Peak CCU'] = peak_players
#     time.sleep(1)  # Pause pour éviter de surcharger le serveur
#     print(f"Mis à jour AppID {appid} avec Peak CCU: {peak_players}")


In [ ]:
# # Enregristrement des données
# jeux_confirmes.to_csv(r"C:\Users\bongu\Documents\GAMEDATA360\data\nettoyes\jeux_confirmes_peak_complet.csv", index = False)
# print("Sauvegarde compl_te reussie !")

In [ ]:
jeux_confirmes.head()

In [ ]:
# Importation de jeux_confirmes_peak_complete
jeux_confirmes_peak_complet = pd.read_csv(r"data/nettoyes/jeux_confirmes_peak_complet.csv")
jeux_confirmes_peak_complet.head()

In [ ]:
# nombre de lignes et de colonnes
jeux_confirmes_peak_complet.shape

In [ ]:
# Maximu de Peak_ccu
max_peak = jeux_confirmes_peak_complet['Peak CCU'].max()
print(f"Le maximum de Peak CCU est: {max_peak}")

In [ ]:
# Affichage du jeux avec le maximum de Peak_ccu
jeux_confirmes_peak_complet[jeux_confirmes_peak_complet['Peak CCU'] == max_peak ]

In [ ]:
# Jeux qui n'on pas de developeur
jeux_sans_devs = jeux_confirmes_peak_complet[jeux_confirmes_peak_complet['Developers'].isnull()]
jeux_sans_devs.head()

In [ ]:
# Scraping des developers depuis Steam Store
import requests
from bs4 import BeautifulSoup

def get_developer_from_steamstore(appid):
    """Retourne le(s) développeur(s) d'un jeu Steam via steamstore."""
    
    url = f"https://store.steampowered.com/app/{appid}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept-Language": "en-US,en;q=0.9"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"❌ Page non trouvée pour AppID {appid}")
            return None
        
        soup = BeautifulSoup(response.text, "html.parser")

        # Chercher le bloc Developers
        dev_block = soup.find("div", id="developers_list")
        if not dev_block:
            print(f"⚠ Aucun développeur trouvé pour AppID {appid}")
            return None
        
        # Récupérer tous les <a> (certains jeux ont plusieurs devs)
        devs = [a.text.strip() for a in dev_block.find_all("a")]
        
        if len(devs) == 0:
            return None
        
        # Si plusieurs développeurs → les joindre avec une virgule
        return ", ".join(devs)

    except Exception as e:
        print(f"🔥 Erreur AppID {appid}: {e}")
        return None


In [ ]:
# Boucle pour scraper les developers pour les jeux sans devs
import time

# → jeux_sans_devs contient déjà seulement ceux qui ont Developers = NaN
for index, row in jeux_sans_devs.iterrows():
    appid = row['AppID']
    
    developer = get_developer_from_steamstore(appid)
    
    # Mise à jour directe dans TON dataframe original
    jeux_confirmes_peak_complet.at[index, 'Developers'] = developer
    
    time.sleep(1)  # respect du serveur
    
    print(f"✔ Mis à jour AppID {appid} avec developer : {developer}")


In [ ]:
# Jeux qui n'ont pas de categories
jeux_sans_categories = jeux_confirmes_peak_complet[jeux_confirmes_peak_complet['Categories'].isnull()]
jeux_sans_categories

In [ ]:
#  Jeux qui n'ont pas de tags
jeux_sans_tags = jeux_confirmes_peak_complet[jeux_confirmes_peak_complet['Tags'].isna()]
jeux_sans_tags.head()

In [ ]:
# Script pour scraper les tags depuis Steam Store
def get_tags_from_steamstore(appid):
    """Retourne la liste des tags d'un jeu Steam depuis Steam Store."""
    
    url = f"https://store.steampowered.com/app/{appid}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept-Language": "fr-FR,fr;q=0.9,en;q=0.8"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"❌ Page non trouvée pour AppID {appid}")
            return None
        
        soup = BeautifulSoup(response.text, "html.parser")

        # Chercher le bloc des tags
        tags_block = soup.find("div", class_="glance_tags")
        if not tags_block:
            print(f"⚠ Aucun tag trouvé pour AppID {appid}")
            return None
        
        # Récupérer tous les <a class="app_tag">
        tag_elements = tags_block.find_all("a", class_="app_tag")
        
        tags = [tag.text.strip() for tag in tag_elements]

        # Si aucun tag trouvé
        if len(tags) == 0:
            return None
        
        # Retourner tags sous forme de chaîne séparée par virgule
        return ", ".join(tags)

    except Exception as e:
        print(f"🔥 Erreur AppID {appid}: {e}")
        return None


In [ ]:
# Boucle pour mettre à jour seulement les jeux sans tags
for index, row in jeux_sans_tags.iterrows():
    appid = row['AppID']
    
    tags = get_tags_from_steamstore(appid)
    
    # Mise à jour directe dans le dataframe complet
    jeux_confirmes_peak_complet.at[index, 'Tags'] = tags
    
    time.sleep(1)  # éviter de spammer Steam
    print(f"✔ Mis à jour AppID {appid} avec tags : {tags}")


In [ ]:
len(jeux_sans_tags)

In [ ]:
# Affichage du jeu Resident Evil 4 Remake
jeux_confirmes_peak_complet[jeux_confirmes_peak_complet['AppID'] == 2050650]

In [ ]:
# Jeux qui n'ont pas de genres
jeux_sans_genres = jeux_confirmes_peak_complet[jeux_confirmes_peak_complet['Genres'].isnull()]
len(jeux_sans_genres)

In [ ]:
# Script pour scraper les genres depuis Steam Store
def get_genres_from_steam(appid):
    """
    Scrape les genres d'un jeu Steam à partir de son AppID.
    Retourne une liste de genres ou None si non trouvé.
    """
    
    url = f"https://store.steampowered.com/app/{appid}"
    
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return None

        soup = BeautifulSoup(response.text, "html.parser")

        genre_spans = soup.select("span[data-panel] a")

        genres = [g.text.strip() for g in genre_spans]
        return genres if genres else None

    except Exception as e:
        print(f"Erreur AppID {appid}: {e}")
        return None


In [ ]:
# Boucle pour mettre à jour seulement les jeux sans genres
for index, row in jeux_sans_genres.iterrows():
    appid = row["AppID"]

    genres = get_genres_from_steam(appid)

    # Met à jour directement dans le DF complet
    jeux_confirmes_peak_complet.at[index, "Genres"] = genres

    print(f"[OK] AppID {appid} → Genres : {genres}")

    time.sleep(1)  # pour éviter le blocage Steam

In [ ]:
# Nombre de jeux sans genres après scraping
len(jeux_sans_genres)

In [ ]:
# Exportatioon de du dataset final avec tous les scrappings
jeux_confirmes_peak_complet.to_csv(r"data/raw/jeux_confirmes_final.csv", index=False)

In [ ]:
# Requette api sur steampowered pour récupérer les scores des jeux par id 
import requests
import time

def get_metascore_from_steam_api(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    try:
        response = requests.get(url)
        data = response.json()
        
        # Vérification si la requête a réussi
        if data and str(app_id) in data and data[str(app_id)]['success']:
            game_data = data[str(app_id)]['data']
            
            # Vérification si le champ metacritic existe
            if 'metacritic' in game_data:
                return game_data['metacritic']['score']
    except Exception as e:
        print(f"Erreur pour {app_id}: {e}")
        
    return None


In [ ]:
# Test d'utilisation 1 
steam_id = 1091500 #(Cyberpunk 2077)
score = get_metascore_from_steam_api(1091500)
print(f"Metascore via Steam : {score}")

In [ ]:
# Test d'utilisation 2
steam_id = 655370 # Train Bandit
score = get_metascore_from_steam_api(steam_id)
print(f"Metascore via Steam : {score}")

In [ ]:
# Test d'utilisation 3
steam_id = 2358720 # wukong
score = get_metascore_from_steam_api(steam_id)
print(f"Metascore via Steam : {score}")

In [ ]:
# Scraping des scores de jeux par leur noms sur metacritic.com
import requests
from bs4 import BeautifulSoup
import re

def clean_game_name(name):
    # Nettoyage : minuscules, remplacer espaces par -, garder alpha-numérique
    name = name.lower()
    name = re.sub(r'[^a-z0-9\s-]', '', name) # Enlever caractères spéciaux (ex: :)
    name = re.sub(r'\s+', '-', name) # Espaces en tirets
    return name

def scrape_metacritic(game_name):
    slug = clean_game_name(game_name)
    url = f"https://www.metacritic.com/game/pc/{slug}" # On suppose que c'est sur PC
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Sélecteur CSS pour le score (peut changer si Metacritic met à jour son site)
            # Souvent c'est une classe du type 'c-productScoreInfo_scoreNumber'
            score_div = soup.find('div', class_=re.compile(r'c-productScoreInfo_scoreNumber'))
            
            if score_div:
                return score_div.text.strip()
    except Exception as e:
        print(f"Erreur scraping {slug}: {e}")
        
    return None

In [ ]:
# Test de fonctionnement
nom_jeu = "Train Bandit"
score = scrape_metacritic(nom_jeu)
print(f"Metascore via Scraping pour {nom_jeu} : {score}")

In [ ]:
# Test de fonctionnement sur Black Myth: Wukong
nom_jeu = "Black Myth: Wukong"
score = scrape_metacritic(nom_jeu)
print(f"Metascore via Scraping pour {nom_jeu} : {score}")

In [ ]:
# Importation du dataset analysis final en vue de mettre à jour les données de score
import pandas as pd
df_analyse = pd.read_csv(r"C:\Users\bongu\Documents\GAMEDATA360\data\nettoyes\jeux_analysis_final.csv")

In [ ]:
# Récupération des jeux dont les scores sont = 0
df_analyse_mscore_0 = df_analyse[
    (df_analyse["Metacritic score"] == 0) | (df_analyse["Metacritic score"].isna())
]

In [ ]:
# Comptage du nombrede de lignes
len(df_analyse_mscore_0)

In [ ]:
# LA BOUCLE D'ITÉRATION
import time
print(f"🔄 Début du scraping pour {len(df_analyse_mscore_0)} jeux...")

count = 0

for index, row in df_analyse_mscore_0.iterrows():
    name = row["Name"]
    
    # Appel de la fonction de scraping
    mscore = scrape_metacritic(name)
    
    if mscore and mscore.isdigit():
        # Met à jour directement dans le DF complet (pas la copie)
        # Assure-toi que 'df_analyse' est bien le nom de ton DF principal
        df_analyse.at[index, "Metacritic score"] = int(mscore)
        
        print(f"[OK] {name} → Metacritic : {mscore}")
        count += 1
    else:
        print(f"[--] {name} → Pas trouvé")

    # SAUVEGARDE INTERMÉDIAIRE (Très important pour le scraping long)
    # Tous les 50 jeux trouvés, on sauvegarde pour ne pas tout perdre si crash
    if count > 0 and count % 50 == 0:
        df_analyse.to_csv("jeux_analysis_enrichi_temp.csv", index=False)
        print("💾 Sauvegarde intermédiaire effectuée.")

    # PAUSE OBLIGATOIRE (Anti-Ban)
    # Metacritic bloque si tu vas trop vite. On met une pause aléatoire entre 2 et 4s.
    time.sleep(1)

print("✅ Terminé !")

In [ ]:
import pandas as pd
from howlongtobeatpy import HowLongToBeat
import time
import random
import ast

# 1. CHARGEMENT
# ---------------------------------------------------------
INPUT_FILE = r"C:\Users\bongu\Documents\GAMEDATA360\data\nettoyes\jeux_analysis_final.csv"
OUTPUT_FILE = r"C:\Users\bongu\Documents\GAMEDATA360\data\nettoyes\jeux_enrichis_hltb.csv"

df = pd.read_csv(INPUT_FILE)

# On nettoie les doublons au cas où
df = df.drop_duplicates(subset=['AppID'])

# On crée les colonnes si elles n'existent pas
if 'HLTB_Main' not in df.columns:
    df['HLTB_Main'] = None
    df['HLTB_Extra'] = None
    df['HLTB_Completionist'] = None

print(f"🚀 Démarrage du scraping HLTB sur {len(df)} jeux...")

# 2. FONCTION DE SCRAPING
# ---------------------------------------------------------
def fetch_hltb_data(game_name):
    hltb = HowLongToBeat()
    try:
        # On cherche le jeu. similarity_case_sensitive=False aide à trouver le bon jeu
        results = hltb.search(game_name, similarity_case_sensitive=False)
        
        if results is not None and len(results) > 0:
            # On prend le résultat avec la meilleure similarité de nom
            # La librairie trie souvent par pertinence, le premier est souvent le bon.
            best_match = max(results, key=lambda x: x.similarity)
            
            # On vérifie que la similarité est décente (ex: > 0.6) pour éviter les faux positifs
            if best_match.similarity > 0.6:
                return {
                    "Main": best_match.gameplay_main,
                    "Extra": best_match.gameplay_main_extra,
                    "Completionist": best_match.gameplay_completionist
                }
    except Exception as e:
        print(f"Erreur sur {game_name}: {e}")
    
    return None

# 3. BOUCLE D'ITÉRATION
# ---------------------------------------------------------
# On filtre pour ne traiter que ceux qui n'ont pas encore été faits (en cas de relance du script)
games_to_process = df[df['HLTB_Main'].isna()]

count = 0

for index, row in games_to_process.iterrows():
    name = row['Name']
    
    data = fetch_hltb_data(name)
    
    if data:
        # Mise à jour du DataFrame
        df.at[index, 'HLTB_Main'] = data['Main']
        df.at[index, 'HLTB_Extra'] = data['Extra']
        df.at[index, 'HLTB_Completionist'] = data['Completionist']
        
        print(f"[OK] {name} -> Story: {data['Main']}h | 100%: {data['Completionist']}h")
    else:
        print(f"[--] {name} -> Pas trouvé sur HLTB")
    
    count += 1
    
    # SAUVEGARDE INTERMÉDIAIRE (Tous les 20 jeux)
    if count % 20 == 0:
        df.to_csv(OUTPUT_FILE, index=False)
        print(f"💾 Sauvegarde intermédiaire ({count} jeux traités)...")
    
    # PAUSE ANTIBAN (Très important pour HLTB qui bloque vite)
    # Pause entre 1.5 et 3 secondes
    time.sleep(random.uniform(1.5, 3))

# 4. SAUVEGARDE FINALE
df.to_csv(OUTPUT_FILE, index=False)
print("✅ Scraping terminé !")